# AutoQuant

This notebook shows a working code example of how to use AIMET AutoQuant feature.

AIMET offers a suite of neural network post-training quantization (PTQ) techniques that can be applied in succession. However, the process of finding the right combination and sequence of techniques to apply is time-consuming and requires careful analysis, which can be challenging especially for non-expert users. We instead recommend AutoQuant to save time and effort.

AutoQuant is an API that applies various PTQ techniques in AIMET automatically based on analyzing the model and best-known heuristics. In AutoQuant, users specify the amount of tolerable accuracy drop, and AutoQuant will apply PTQ techniques cumulatively until the target accuracy is satisfied.


#### Overall flow
This notebook covers the following
1. Define constants and helper functions
2. Load a pretrained FP32 model
3. Run AutoQuant

#### What this notebook is not
This notebook is not designed to show state-of-the-art AutoQuant results. For example, it uses a relatively quantization-friendly model like Resnet18. Also, some optimization parameters are deliberately chosen to have the notebook execute more quickly.


---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification. If you already have a version of the dataset readily available, please use that. Else, please download the dataset from appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The ImageNet dataset typically has the following characteristics and the dataloader provided in this example notebook rely on these
- Subfolders 'train' for the training samples and 'val' for the validation samples. Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset. E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. But for the purpose of running this notebook, you could perhaps reduce the dataset to say 2 samples per class. This exercise is left upto the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [1]:
import cv2
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import json
from mmcv.transforms import Compose
import numpy as np
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            transformed_images = self.transform(image)
        else:
            transformed_images = image

        return image_id, image_path, transformed_images


# calibrationDataloader = DataLoader(calibrationDataset, batch_size=32, shuffle=True)

## 1. Define Constants and Helper functions

In this section the constants and helper functions needed to run this eaxmple are defined.

- **EVAL_DATASET_SIZE** A typical value is 5000. In this notebook, this value has been set to 500 for faster execution.
- **CALIBRATION_DATASET_SIZE** A typical value is 2000. In this notebook, this value has been set to 200 for faster execution.


The helper function **_create_sampled_data_loader()** returns a DataLoader based on the dataset and the number of samples provided.

## 2. Load a pretrained FP32 model
For this example, we are going to load a pretrained resnet18 model from torchvision. Similarly, you can load any pretrained PyTorch model instead.

In [2]:
# %cd /content/drive/MyDrive/Aimet-torch/mmdetection-3.3.0/
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = 'rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = 'rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

ROOT_DATASET_DIR = '/datasets/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH)
evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)


2024-08-02 16:04:40,428 - root - INFO - AIMET
Loads checkpoint by local backend from path: rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

08/02 16:04:44 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/home/shayaan/miniconda3/envs/aimet2/lib/python3.10/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [3]:
from mmcv.transforms import Compose
test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
import random
from typing import Optional
from tqdm.notebook import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from mmengine.structures import InstanceData

EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 2000
BATCH_SIZE = 64
# _datasets = {}

# def _create_sampled_data_loader(dataset, num_samples):
#     if num_samples not in _datasets:
#         indices = random.sample(range(len(dataset)), num_samples)
#         _datasets[num_samples] = Subset(dataset, indices)
#     return DataLoader(_datasets[num_samples], batch_size=BATCH_SIZE)


def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:    
    data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)

    new_preds = []
    for image_id, image_path, _ in tqdm(data_loader):
        preds = model(image_path, out_dir='./output')
        
        for img_id, pred in zip(image_id, preds['predictions']):
            new_pred = InstanceData(metainfo={"img_id": int(img_id)})
            new_pred.bboxes = [np.array(p) for p in pred['bboxes']]
            new_pred.labels = pred['labels']
            new_pred.scores = pred['scores']
            new_preds.append(new_pred)

    eval_results = test_evaluator(new_preds)
    with open("/home/shayaan/Desktop/aimet/my_mmdet/ptq_stats/eval_acc_fp32.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    bbox_map = eval_results['bbox_mAP']
    return bbox_map

In [5]:
from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
from aimet_torch.model_preparer import prepare_model

dummy_input = torch.rand(1, 3, 640, 640)


sim_model = prepare_model(model.model)

sim = QuantizationSimModel(model=sim_model,
                        quant_scheme=QuantScheme.post_training_tf_enhanced,
                        dummy_input=dummy_input,
                        default_output_bw=8,
                        default_param_bw=8,)

2024-08-02 16:04:44,558 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.0.bn.module_batch_norm} 
2024-08-02 16:04:44,559 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.1.bn.module_batch_norm_1} 
2024-08-02 16:04:44,559 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.2.bn.module_batch_norm_2} 
2024-08-02 16:04:44,560 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.0.bn.module_batch_norm_3} 
2024-08-02 16:04:44,560 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.short_conv.bn.module_batch_norm_4} 
2024-08-02 16:04:44,560 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.main_conv.bn.module_batch_norm_5} 
2024-08-02 16:04:44,561 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.blocks.0.conv1.bn.

In [6]:
# accuracy = eval_callback(model)

In [7]:
# print(f'- FP32 accuracy: {accuracy}')

## 3. Run AutoQuant
### Create AutoQuant Object

The AutoQuant feature utilizes an unlabeled dataset to achieve quantization. The class **UnlabeledDatasetWrapper** creates an unlabeled Dataset object from a labeled Dataset.

In [8]:
class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset, num_samples):
        self._dataset = dataset
        self.num_samples = num_samples if num_samples < len(self._dataset) else len(self._dataset)
        self.transform = transform

    def __len__(self):
        return self.num_samples 

    def __getitem__(self, index):
        _, _, transformed_images = self._dataset[index]
        return transformed_images
    

In [9]:
preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True
from mmdet.models.utils import samplelist_boxtype2tensor
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

In [10]:
from aimet_torch.auto_quant import AutoQuant

def new_eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)
    new_preds = []
    for image_id, image_path, _ in tqdm(data_loader):
        pre_processed = preprocessor(inputs=image_path, batch_size=BATCH_SIZE)
        _, data = list(pre_processed)[0]
        inp_image = np.ndarray(shape=(BATCH_SIZE, 3, 640, 640))
        for idx, img in enumerate(data['inputs']):
            inp_image[idx] = img
        inp_image = torch.from_numpy(inp_image).float()
        preds = model(inp_image)
        batch_img_metas = [
        data_samples.metainfo for data_samples in data['data_samples']
        ]
        preds = predict_by_feat(*preds, batch_img_metas=batch_img_metas, rescale=True)
        preds = add_pred_to_datasample(data['data_samples'], preds)
        
        for img_id, pred in zip(image_id, preds):
            pred = pred.pred_instances
            new_pred = InstanceData(metainfo={"img_id": int(img_id)})
            new_pred.bboxes = [np.array(p) for p in pred['bboxes']]
            new_pred.labels = pred['labels']
            new_pred.scores = pred['scores']
            new_preds.append(new_pred)

        # new_pred = InstanceData(metainfo={"img_id": int(image_id)})
        # new_pred.bboxes = [np.array(p) for p in preds['bboxes']]
        # new_pred.labels = preds['labels']
        # new_pred.scores = preds['scores']
        # new_preds.append(new_pred)

    eval_results = test_evaluator(new_preds)
    from glob import glob
    num_file = len(glob("/home/shayaan/Desktop/aimet/my_mmdet/ptq_stats/eval_acc_quant_*"))
    with open(f"/home/shayaan/Desktop/aimet/my_mmdet/ptq_stats/eval_acc_quant_{num_file}.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    bbox_map = eval_results['bbox_mAP']
    return bbox_map


calibration_data_loader = DataLoader(UnlabeledDatasetWrapper(evalDataset, num_samples=CALIBRATION_DATASET_SIZE))
auto_quant = AutoQuant(sim_model,
                       dummy_input=dummy_input,
                       data_loader=calibration_data_loader,
                       eval_callback=new_eval_callback)

### Run AutoQuant Inference
This step runs AutoQuant inference. AutoQuant inference will run evaluation using the **eval_callback** with the vanilla quantized model without applying PTQ techniques. This will be useful for measuring the baseline evaluation score before running AutoQuant optimization.

In [11]:
sim, initial_accuracy = auto_quant.run_inference()


2024-08-02 16:04:49,609 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm} 


2024-08-02 16:04:49,611 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul} 


2024-08-02 16:04:49,612 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_1} 


2024-08-02 16:04:49,613 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul_1} 


2024-08-02 16:04:49,614 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_2} 


...

2024-08-02 16:05:13,280 - Quant - WARNING - The following layers were not found in the exported onnx model. Encodings for these layers will not appear in the exported encodings file:
['backbone.stem.0.conv', 'backbone.stem.0.activate.sigmoid', 'backbone.stem.1.conv', 'backbone.stem.1.activate.sigmoid', 'backbone.stem.2.conv', 'backbone.stem.2.activate.sigmoid', 'backbone.stage1.0.conv', 'backbone.stage1.0.activate.sigmoid', 'backbone.stage1.1.short_conv.conv', 'backbone.stage1.1.short_conv.activate.sigmoid', 'backbone.stage1.1.main_conv.conv', 'backbone.stage1.1.main_conv.activate.sigmoid', 'backbone.stage1.1.blocks.0.conv1.conv', 'backbone.stage1.1.blocks.0.conv1.activate.sigmoid', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.conv', 'backbone.stage1.1.blocks.0.conv2.depthwise_conv.activate.sigmoid', 'backbone.stage1.1.blocks.0.conv2.pointwise_conv.conv', 'backbone.stage1.1.blocks.0.conv2.pointwise_conv.activate.sigmoid', 'backbone.stage1.1.attention.global_avgpool', 'backbone.stag

2024-08-02 16:05:13,281 - Quant - INFO - Layers excluded from quantization: []


2024-08-02 16:05:13,286 - AutoQuant - INFO - The results of Batchnorm Folding is saved in /tmp/.trace/batchnorm_folding.pth and /tmp/.trace/batchnorm_folding.encodings.


2024-08-02 16:05:13,287 - AutoQuant - INFO - Session finished: Batchnorm Folding. (eval score: 0.000000)


- Batchnorm Folding


In [ ]:
print(f"- Quantized A ccuracy (before optimization): {initial_accuracy}")

### Set AdaRound Parameters (optional)
AutoQuant uses a set of predefined default parameters for AdaRound.
These values were determined empirically and work well with the common models.
However, if necessary, you can also use your custom parameters for Adaround.
In this notebook, we will use very small AdaRound parameters for faster execution.

In [ ]:
from aimet_torch.adaround.adaround_weight import AdaroundParameters

adaround_params = AdaroundParameters(calibration_data_loader, num_batches=len(calibration_data_loader), default_num_iterations=2000)
auto_quant.set_adaround_params(adaround_params)

### Run AutoQuant Optimization
This step runs AutoQuant optimization, which returns the best possible quantized model, corresponding evaluation score and the path to the encoding file.
The **allowed_accuracy_drop** parameter indicates the tolerable amount of accuracy drop. AutoQuant applies a series of quantization features until the target accuracy (FP32 accuracy - allowed accuracy drop) is satisfied. When the target accuracy is reached, AutoQuant will return immediately without applying furhter PTQ techniques. Please refer AutoQuant User Guide and API documentation for complete details.

In [ ]:
model, optimized_accuracy, encoding_path = auto_quant.optimize(allowed_accuracy_drop=0.01)
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy}")

In [ ]:
auto_quant.dummy_input.shape

from aimet_torch.quantsim import QuantizationSimModel

model = QuantizationSimModel(model, auto_quant.dummy_input)

---
## Summary

Hope this notebook was useful for you to understand how to use AIMET AutoQuant feature.

Few additional resources
- Refer to the AIMET API docs to know more details of the APIs and parameters
- Refer to the other example notebooks to understand how to use AIMET CLE and AdaRound features in a standalone fashion.